# [xTuring](https://github.com/stochasticai/xturing) - LLaMA INT4 efficient fine-tuning tutorial

This tutorial aims to show how easy it is to perform fine-tuning with xTuring. This notebook shows how to fine-tune LLaMA 7B model on GPU which has limited memory, it requires only 6GB VRAM

## 1. Install the `xTuring` library

In [ ]:
!pip install xturing --upgrade

## 2. Load model and dataset

In [ ]:
from xturing.datasets.instruction_dataset import InstructionDataset
from xturing.models import BaseModel

instruction_dataset = InstructionDataset("../llama/alpaca_data")
# Initializes the model
model = BaseModel.create("llama_lora_int4")

## 3. Start the finetuning

In [ ]:
# Finetuned the model
model.finetune(dataset=instruction_dataset)

## 4. Generate an output text with the fine-tuned model

In [ ]:
# Once the model has been finetuned, you can start doing inferences
output = model.generate(texts=["Why LLM models are becoming so important?"])
print("Generated output by the model: {}".format(output))